In [1]:
import numpy as np
import pandas as pd

import argparse
import datetime as dte
import os
pd.set_option('display.max_columns', None)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import medical_rl.libs.cluster as cluster
import medical_rl.libs.envs as envs
pd.set_option('display.max_columns', None)
import medical_rl.libs.utils as utils
import medical_rl.data_formatters.boston


name, output_folder =  'boston', '/gpfs/data/fs71801/lkapral66/RL4RRT/RRT_mimic_iv/data/cluster'


data_formatter = medical_rl.data_formatters.boston.BostonFormatter()

expt_name=name



if not isinstance(data_formatter, medical_rl.data_formatters.base.GenericDataFormatter):
    raise ValueError(
        "Data formatters should inherit from" +
        "AbstractDataFormatter! Type={}".format(type(data_formatter)))



print("*** Training from defined parameters for {} ***".format(expt_name))

print("Loading & splitting data...")
df = pd.read_parquet('/gpfs/data/fs71801/lkapral66/RL4RRT/RRT_mimic_iv/data/mimiciv/MIMIC_action.parquet')





*** Training from defined parameters for boston ***
Loading & splitting data...


In [4]:
number_of_clusters = 750
n_init = 'auto'
seed=42

In [16]:
len(df[df['action']>=1]['icustayid'].unique())

3998

In [17]:
len(df['icustayid'].unique())

70080

In [5]:
import medical_rl.libs.cluster as cluster
import medical_rl.libs.envs as envs

In [6]:
#data_formatter.save_random_splits(df, 5, output_folder, train_size=0.7, valid_size=0.1, test_size=0.2)

In [9]:
train, valid, test, bounds = data_formatter.load_random_split_and_bounds(output_folder, 42, train_size=0.7, valid_size=0.1, test_size=0.2, parse_bounds=False)

In [10]:
list(test.columns)

['icustayid',
 'bloc',
 'extubated',
 'max_dose_vaso',
 'age',
 'Height_cm',
 'Weight_kg',
 'MeanBP',
 'Potassium',
 'Arterial_pH',
 'paO2',
 'Arterial_BE',
 'Arterial_lactate',
 'SvO2',
 'Anion_Gap',
 'Absolute_Neutrophil_Count',
 'SOFA',
 'cumulated_balance',
 'Sodium_Urine',
 'BUN',
 'Creatinine',
 'CRP',
 'output_total',
 'input_total',
 'input_step',
 'output_step',
 'gender',
 'action',
 'morta_90']

In [13]:
obs_column_labels = [
 'extubated',
 'max_dose_vaso',
 'age',
 'Height_cm',
 'Weight_kg',
 'MeanBP',
 'Potassium',
 'Arterial_pH',
 'paO2',
 'Arterial_BE',
 'Arterial_lactate',
 'SvO2',
 'Anion_Gap',
 'Absolute_Neutrophil_Count',
 'SOFA',
 'cumulated_balance',
 'Sodium_Urine',
 'BUN',
 'Creatinine',
 'CRP',
 'output_total',
 'input_total',
 'input_step',
 'output_step',
 'gender']

In [14]:
def get_column_labels():
    return {
        "id": "icustayid",
        "t":  "bloc",
        "obs": obs_column_labels,
        "act": "action",
        "rew": "morta_90",}


In [15]:
column_labels = get_column_labels()

In [16]:

from medical_rl.libs.cluster import get_data_clustered_completed

train_clustered = get_data_clustered_completed(
    data=train, column_labels=column_labels,
    n_clusters=number_of_clusters, n_init=n_init, seed=seed, path=output_folder,
)


Trying to load /gpfs/data/fs71801/lkapral66/RL4RRT/RRT_mimic_iv/data/cluster/kmeans_model.pkl


In [17]:


test_clustered = get_data_clustered_completed(
    data=test, column_labels=column_labels,
    n_clusters=number_of_clusters, n_init=n_init, seed=seed, path=output_folder,
)


Trying to load /gpfs/data/fs71801/lkapral66/RL4RRT/RRT_mimic_iv/data/cluster/kmeans_model.pkl


In [19]:
test_clustered

,id,t,obs,act,rew
0,30003226,1,549,0,0
1,30003226,2,711,0,0
2,30003226,3,223,0,0
3,30003226,4,223,0,0
4,30003226,5,711,1,0
...,...,...,...,...,...
51821,37148034,2,139,0,0
51822,37148034,3,246,0,0
51823,37148034,4,246,0,0
51824,37148034,5,246,0,0


In [20]:
env, action_mask = envs.get_env(number_of_clusters, train_clustered, n_act=2)


In [32]:
env.step(0)

(750, 0, True, False, {'absorbing': True})

In [22]:
from sb3_contrib import MaskablePPO

In [24]:

def load_or_create_model(
        model_type,
        model_dir, env=None, total_timesteps=None):

    try:
        print("Try loading model", model_dir)
        model = model_type.load(model_dir)

    except KeyboardInterrupt:
        print("KeyboardInterrupt")
        assert False

    except:
        assert env is not None
        assert total_timesteps is not None

        print(); print(f"No model found in {model_dir}")

        model = model_type("MlpPolicy", env, verbose=1)

        model.learn(total_timesteps=total_timesteps, tb_log_name="ppo_run")
        model.save(model_dir)

    return model

def load_or_create_model_MaskablePPO(model_dir, env=None, total_timesteps=None):
    return load_or_create_model(MaskablePPO, model_dir, env, total_timesteps)


In [25]:
model_path = os.path.join(output_folder, 'ppo.zip')

In [ ]:
model = load_or_create_model(MaskablePPO, model_path, env, 100000)

In [ ]:
output_folder